In [1]:
import sys, os
sys.path.append('./boolODE/')

import ast
import yaml
import time
import warnings
import numpy as np
import pandas as pd
from tqdm.auto import tqdm 
from pathlib import Path
import multiprocessing as mp
from optparse import OptionParser

from scipy.integrate import odeint
from sklearn.cluster import KMeans
from typing import Dict, List
from importlib.machinery import SourceFileLoader
from scipy.stats import rankdata
import copy
import re

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

import parsing_fnc as fnc
import gen_graph_util as util

In [2]:
# Load initial graph (load long-linear, but change topology to 1:n regulation as initial graph)
# Graph statistics (nodes = 18, edges = 18)

path = "./"

# Manually load
# grn_init = path + "graph_LL_18"
# df = pd.read_csv(grn_init + ".txt", sep="\t", engine="python")

node_size = 18
genes = {"Gene":["g"+str(g) for g in np.arange(node_size)], "Rule":["g"+str(g) for g in np.arange(node_size)]}
df = pd.DataFrame(data = genes)

In [3]:
# Random sample transripciton factors and build initail topology
ntfs = 6
# select the tfs
tfs = np.random.choice(df.values[:,0], ntfs, replace = False)

for i in range(len(df)):
    # choose the TF for gene i
    tf = tfs[int(i%ntfs)]
    
    if np.random.choice(2) == 0:
        df.values[:,1][i] = "( " + tf + " )"
    else:
        df.values[:,1][i] = "( not ( " + tf + " ) )"

# df is the dataframe for bool grn        
genes, all_regul, activate_dict, inactivate_dict = util.load_init_graph(df)

In [4]:
# Set kinetic parameters (use same parameters used in BEELINE)
withRules = list(df['Gene'].values)
allnodes = set()

# allnodes stores all tfs used
for ind, row in df.iterrows():
    rhs = row['Rule']
    rhs = rhs.replace('(',' ')
    rhs = rhs.replace(')',' ')
    tokens = rhs.split(' ')
    reg = [t for t in tokens if t not in ['not','and','or','']]
    allnodes.update(set(reg))
    
# filter gene which has no regulate rules
withoutRules = list(allnodes.difference(set(withRules)))

# if no rule, assign self-activation, make sure withoutRules is empty
for n in withoutRules:
    print(n, "has no rule, adding self-activation.")
    df = df.append({'Gene':n,'Rule':n}, ignore_index=True)
    withRules.append(n)
    withoutRules.remove(n)
    
# Assume everything is a gene, so make the corresponding protein
varspecs, genelist, inputs = dict(), list(), list()
for node in withRules:
    varspecs['x_' + node] = ''
    varspecs['p_' + node] = ''
    genelist.append(node)

kineticParameterDefaults = {'mRNATranscription':20.,'mRNADegradation':10.,'proteinTranslation':10.,'proteinDegradation':1.0,
                            'heavisideSigma':10.,'signalingTimescale':5.0,'hillCoefficient':10.,'interactionStrength':1.0}

# Max level checks, 
# the maximum gene expression level, this is achieved by dx/dt = M * 1 - D * x, where the solution is M/D * (1 - e^-t)
x_max = kineticParameterDefaults['mRNATranscription']/kineticParameterDefaults['mRNADegradation']
y_max = x_max*(kineticParameterDefaults['proteinTranslation']/kineticParameterDefaults['proteinDegradation'])

# set hill threshold to be a half of the maximum expression of protein
hillThreshold = y_max/2
heavisideOmega = 2./y_max

kineticParameterDefaults['x_max'] = x_max
kineticParameterDefaults['y_max'] = y_max
kineticParameterDefaults['hillThreshold'] = hillThreshold
kineticParameterDefaults['heavisideOmega'] = heavisideOmega

# for all genes, the hill coefficient (n_) = 10 (kineticParameterDefaults), the hill threshold (k_) = y_max/2
parameterNamePrefixAndDefaultsAll = {'n_':kineticParameterDefaults['hillCoefficient'], 'k_':hillThreshold, 
                                     'sigmaH_':kineticParameterDefaults['heavisideSigma']}
# for all genes, set the mRNA transcription rate (m_) to be 20, degradation rate (l_x_) to be 10, etc
parameterNamePrefixAndDefaultsGenes = {'m_':kineticParameterDefaults['mRNATranscription'],'l_x_':kineticParameterDefaults['mRNADegradation'],
                                       'r_':kineticParameterDefaults['proteinTranslation'],'l_p_':kineticParameterDefaults['proteinDegradation']}

proteinlist, interactionStrengths = list(), dict()
parameterSetDF, parameterInputsDF = pd.DataFrame(), pd.DataFrame()

In [5]:
# Setting experiments (activation function, timelength, cell number)

settings = dict()
settings['modeltype'] = 'hill'
settings['num_cells'] = 3
settings['simulation_time'] = 75
settings['integration_step_size'] = 0.01
settings['doParallel'] = False

print("Simulation Time Length: ", int(settings['simulation_time']/settings['integration_step_size']))

# par include the parameter of hill function (m, n, k, l) of all genes
# TODO: par (k_) should be different and should be related between genes
par = fnc.assignDefaultParameterValues(parameterNamePrefixAndDefaultsAll,parameterNamePrefixAndDefaultsGenes,withRules, genelist)

Simulation Time Length:  7500
Fixing rate parameters to defaults


In [6]:
par

{'n_g0': 10.0,
 'n_g1': 10.0,
 'n_g2': 10.0,
 'n_g3': 10.0,
 'n_g4': 10.0,
 'n_g5': 10.0,
 'n_g6': 10.0,
 'n_g7': 10.0,
 'n_g8': 10.0,
 'n_g9': 10.0,
 'n_g10': 10.0,
 'n_g11': 10.0,
 'n_g12': 10.0,
 'n_g13': 10.0,
 'n_g14': 10.0,
 'n_g15': 10.0,
 'n_g16': 10.0,
 'n_g17': 10.0,
 'k_g0': 10.0,
 'k_g1': 10.0,
 'k_g2': 10.0,
 'k_g3': 10.0,
 'k_g4': 10.0,
 'k_g5': 10.0,
 'k_g6': 10.0,
 'k_g7': 10.0,
 'k_g8': 10.0,
 'k_g9': 10.0,
 'k_g10': 10.0,
 'k_g11': 10.0,
 'k_g12': 10.0,
 'k_g13': 10.0,
 'k_g14': 10.0,
 'k_g15': 10.0,
 'k_g16': 10.0,
 'k_g17': 10.0,
 'sigmaH_g0': 10.0,
 'sigmaH_g1': 10.0,
 'sigmaH_g2': 10.0,
 'sigmaH_g3': 10.0,
 'sigmaH_g4': 10.0,
 'sigmaH_g5': 10.0,
 'sigmaH_g6': 10.0,
 'sigmaH_g7': 10.0,
 'sigmaH_g8': 10.0,
 'sigmaH_g9': 10.0,
 'sigmaH_g10': 10.0,
 'sigmaH_g11': 10.0,
 'sigmaH_g12': 10.0,
 'sigmaH_g13': 10.0,
 'sigmaH_g14': 10.0,
 'sigmaH_g15': 10.0,
 'sigmaH_g16': 10.0,
 'sigmaH_g17': 10.0,
 'm_g0': 20.0,
 'm_g1': 20.0,
 'm_g2': 20.0,
 'm_g3': 20.0,
 'm_g4': 20.0,
 

In [7]:
ModelSpecs, varmappers, parmappers, model_path = dict(), dict(), dict(), dict()
time_length = int(settings['simulation_time']/settings['integration_step_size'])

In [8]:
# Set perturb rulse for ground-truth graphs
pert_method = "swap"

btw_graphs = 1500
num_graphs, num_pert = time_length//btw_graphs, 5

print("Num Graphs: {}, Num Pert: {}".format(num_graphs,num_pert))
sub_path = path + pert_method + "_time_" + str(time_length) + "_graphs_" + str(num_graphs) + "(" + str(num_pert) + ")_1to2/"

simgraphpath = Path(sub_path + "./graphs/")
if not os.path.exists(simgraphpath):
    os.makedirs(simgraphpath)
    
simmodelpath = Path(sub_path + "./models/")
if not os.path.exists(simmodelpath):
    os.makedirs(simmodelpath)

Num Graphs: 5, Num Pert: 5


In [10]:
import importlib
importlib.reload()

[]

In [23]:
# Gen ground-truth graphs
for grn_num in range(num_graphs):

    # df saves the binary regulation rules (boolean network) gene\t bool regulation
    if grn_num == 0: # initial graph_0
        df.to_csv(sub_path + "graphs/graph_"+str(grn_num)+".txt",header=True, index=False, sep="\t")

    if grn_num > 0:
        rows = np.random.choice(genes, num_pert*2, replace = False) - 1
        df.values[:,1][rows] = df.values[:,1][rows[::-1]]
        df.to_csv(sub_path + "graphs/graph_"+str(grn_num)+".txt",header=True, index=False, sep="\t")

    # df stores the regulation relationship, setting is the generall setting of the simulator, inputs?? par is the parameter for hill function, genelist and proteinlist
    # varspecs is the disctionary of regulation relationship, empty for each gene and protein 
    ModelSpec, varmapper, parmapper = util.model_generate(df, settings, withRules, inputs, par, genelist, proteinlist, varspecs)

    ModelSpecs[grn_num]=copy.deepcopy(ModelSpec)
    varmappers[grn_num]=copy.deepcopy(varmapper)
    parmappers[grn_num]=copy.deepcopy(parmapper)

    model_path[grn_num] = fnc.writeModelToFile(grn_num, ModelSpec, varmapper, path = sub_path + "models/")
    fnc.generateInputFiles(df, withoutRules, grn_num, path = sub_path)

In [26]:
varmapper

{0: 'x_g0',
 1: 'p_g0',
 2: 'x_g1',
 3: 'p_g1',
 4: 'x_g2',
 5: 'p_g2',
 6: 'x_g3',
 7: 'p_g3',
 8: 'x_g4',
 9: 'p_g4',
 10: 'x_g5',
 11: 'p_g5',
 12: 'x_g6',
 13: 'p_g6',
 14: 'x_g7',
 15: 'p_g7',
 16: 'x_g8',
 17: 'p_g8',
 18: 'x_g9',
 19: 'p_g9',
 20: 'x_g10',
 21: 'p_g10',
 22: 'x_g11',
 23: 'p_g11',
 24: 'x_g12',
 25: 'p_g12',
 26: 'x_g13',
 27: 'p_g13',
 28: 'x_g14',
 29: 'p_g14',
 30: 'x_g15',
 31: 'p_g15',
 32: 'x_g16',
 33: 'p_g16',
 34: 'x_g17',
 35: 'p_g17'}

In [24]:
ModelSpec

{'varspecs': {'x_g0': 'm_g0*(( alpha_g0 + a_g0_g16*(p_g16/k_g16)^n_g16 )/( 1 +(p_g16/k_g16)^n_g16 ))-l_x_g0*x_g0',
  'p_g0': 'r_g0*x_g0- l_p_g0*p_g0',
  'x_g1': 'm_g1*(( alpha_g1 + a_g1_g11*(p_g11/k_g11)^n_g11 )/( 1 +(p_g11/k_g11)^n_g11 ))-l_x_g1*x_g1',
  'p_g1': 'r_g1*x_g1- l_p_g1*p_g1',
  'x_g2': 'm_g2*(( alpha_g2 + a_g2_g4*(p_g4/k_g4)^n_g4 )/( 1 +(p_g4/k_g4)^n_g4 ))-l_x_g2*x_g2',
  'p_g2': 'r_g2*x_g2- l_p_g2*p_g2',
  'x_g3': 'm_g3*(( alpha_g3 + a_g3_g4*(p_g4/k_g4)^n_g4 )/( 1 +(p_g4/k_g4)^n_g4 ))-l_x_g3*x_g3',
  'p_g3': 'r_g3*x_g3- l_p_g3*p_g3',
  'x_g4': 'm_g4*(( alpha_g4 + a_g4_g2*(p_g2/k_g2)^n_g2 )/( 1 +(p_g2/k_g2)^n_g2 ))-l_x_g4*x_g4',
  'p_g4': 'r_g4*x_g4- l_p_g4*p_g4',
  'x_g5': 'm_g5*(( alpha_g5 + a_g5_g11*(p_g11/k_g11)^n_g11 )/( 1 +(p_g11/k_g11)^n_g11 ))-l_x_g5*x_g5',
  'p_g5': 'r_g5*x_g5- l_p_g5*p_g5',
  'x_g6': 'm_g6*(( alpha_g6 + a_g6_g16*(p_g16/k_g16)^n_g16 )/( 1 +(p_g16/k_g16)^n_g16 ))-l_x_g6*x_g6',
  'p_g6': 'r_g6*x_g6- l_p_g6*p_g6',
  'x_g7': 'm_g7*(( alpha_g7 + a_g7_g

In [10]:
# Initialize Expression data - using first Graph (graph_0)
init_Model = ModelSpecs[0]
init_varmap = varmappers[0]
init_parmap = parmappers[0]

####################
rnaIndex = [i for i in range(len(init_varmap.keys())) if 'x_' in init_varmap[i]]
revvarmapper = {v:k for k,v in init_varmap.items()}
proteinIndex = [i for i in range(len(init_varmap.keys())) if 'p_' in init_varmap[i]]

y0 = [ModelSpec['ics'][init_varmap[i]] for i in range(len(init_varmap.keys()))]
ss = np.zeros(len(init_varmap.keys()))

for i,k in init_varmap.items():
    if 'x_' in k:
        ss[i] = 1.0
    elif 'p_' in k:
        if k.replace('p_','') in proteinlist:
            ss[i] = 20.

In [11]:
tmax = settings['simulation_time']
integration_step_size = settings['integration_step_size']
tspan = np.linspace(0,tmax,int(tmax/integration_step_size))

In [12]:
#Load initial expression values
init_exp = {"Genes":["['g1']"], "Values":"[1]"}
icsDF = pd.DataFrame(data=init_exp)

if not icsDF.empty:
    icsspec = icsDF.loc[0]
    genes = ast.literal_eval(icsspec['Genes'])
    values = ast.literal_eval(icsspec['Values'])
    icsmap = {g:v for g,v in zip(genes,values)}
    for i,k in init_varmap.items():
        for g in genelist:
            if g in icsmap.keys():
                ss[revvarmapper['x_'+g]] = icsmap[g]
            else:
                ss[revvarmapper['x_'+g]] = 0.01

result = pd.DataFrame(index=pd.Index([varmapper[i] for i in rnaIndex]))

In [13]:
# Index of every possible time point. Sample from this list
startat = 0
timeIndex = [i for i in range(startat, len(tspan))]

groupedDict = {}

simfilepath = Path(sub_path + "./simulations/")
if not os.path.exists(simfilepath):
    os.makedirs(simfilepath)

In [14]:
# Initialize
new_ics = [0 for _ in range(len(varmapper.keys()))]

# Set the mRNA ics
for ind in rnaIndex:
    if ss[ind] < 0:
        ss[ind] = 0.0
    new_ics[ind] =  ss[ind]
    if new_ics[ind] < 0:
        new_ics[ind] = 0
for p in proteinlist:
    ind = revvarmapper['p_'+p]
    if ss[ind] < 0:
        ss[ind] = 0.0
    new_ics[ind] =  ss[ind]
    if new_ics[ind] < 0:
        new_ics[ind] = 0
        
# Calculate the Protein ics based on mRNA levels
for genename in genelist:
    pss = ((ModelSpec['pars']['r_' + genename])/(ModelSpec['pars']['l_p_' + genename]))*new_ics[revvarmapper['x_' + genename]]
    new_ics[revvarmapper['p_' + genename.replace('_','')]] = pss
    
argdict = {}
argdict['Model'] = model_path
argdict['tspan'] = tspan
argdict['varmapper'] = init_varmap
argdict['timeIndex'] = timeIndex
argdict['genelist'] = genelist
argdict['proteinlist'] = proteinlist
argdict['ss'] = ss
argdict['ModelSpecs'] = ModelSpecs
argdict['parmappers'] = parmappers
argdict['rnaIndex'] = rnaIndex
argdict['proteinIndex'] = proteinIndex
argdict['revvarmapper'] = revvarmapper
argdict['x_max'] = kineticParameterDefaults['x_max']

if settings['doParallel']:
    with mp.Pool() as pool:
        jobs = []
        for cellid in range(settings['num_cells']):
            cell_args = dict(argdict, seed=cellid, cellid=cellid)
            job = pool.apply_async(fnc.simulateAndSample, args=(cell_args,new_ics))
            jobs.append(job)

        for job in jobs:
            job.wait()
# not doing it in a parallel manner
else:
    # for each cellid, give a new experiment with new seed.
    for cellid in tqdm(range(settings['num_cells'])):
        argdict['seed'] = cellid
        argdict['cellid'] = cellid
        fnc.simulateAndSample(argdict, new_ics, path = sub_path)

100%|██████████| 3/3 [00:04<00:00,  1.61s/it]


In [15]:
# # after running the simulation, sample cells, print expression and pseudotime
# frames = []
# print('starting to concat files')
# for cellid in tqdm(range(settings['num_cells'])):
#     # cellid correspond to the cellid-th experiment, read in the gene expression simulation for the cellid-th experiment
#     df = pd.read_csv(sub_path + './simulations/E'+str(cellid) + '.csv',index_col=0)
#     # df of the form genes by times
#     df = df.sort_index()
#     # times by genes
#     frames.append(df.T)
    
# # experiment * times by genes, 
# result = pd.concat(frames,axis=0)
# result = result.T

# # indices are gene names
# indices = result.index
# newindices = [i.replace('x_','') for i in indices]
# result.index = pd.Index(newindices)

In [16]:
# Re-organizing generated simulation data: 1. cell expression data 2. ground-truth adjacent matrix
# consider the first single cell trajectory

exp_data = pd.read_csv(sub_path + "simulations/E0.csv", index_col = 0) 

_, ntime_1 = exp_data.shape
time_len = ntime_1 + 1

if exp_data.shape[1] != time_len:
    (cell, time) = exp_data.columns[-1].split('_')
    exp_data[cell+'_'+str(int(time)+1)]=exp_data[cell+'_'+time]
    
# Sort based on time - order
sorted_exp = np.array(exp_data)[:,np.argsort([int(col.split("_")[1]) for col in exp_data.columns])]

temp = re.compile("([a-zA-Z]+)([0-9]+)")

gene_dict = dict()
gene_list = [int(gene.split("g")[1]) for gene in exp_data.index]
ngenes = len(gene_list)
gene_rank = rankdata(gene_list, method="min")

for gene, rank in zip(gene_list, gene_rank):
    gene_dict[gene] = rank

ref_net = dict()
n_graphs = len(os.listdir(sub_path + "ground_truth"))

gt_adj = np.zeros((n_graphs,ngenes,ngenes))
for i in range(n_graphs):
    ref_net = pd.read_csv(sub_path + "ground_truth/refNetwork_" +str(i) + ".csv", index_col = 0)
    target = list(ref_net.index)
    regula = list(ref_net.values[:,0])

    target = [int(temp.match(idx).groups()[1]) for idx in target]
    regula = [int(temp.match(idx).groups()[1]) for idx in regula]
    
    target = [gene_dict[i] for i in target]
    regula = [gene_dict[i] for i in regula]
    start_idx = min(gene_rank)
    
    for rule in range(len(target)):
        node0, node1 = target[rule], regula[rule]
        gt_adj[i, node0 - start_idx, node1 - start_idx] += 1

gt_adj_time = np.repeat(gt_adj,btw_graphs,axis = 0)

if btw_graphs > 10:
    freq = "discrete_"
else:
    freq = "continue_"

outpath = "../../data/boolODE_Sep13/"
    
np.save(outpath+freq+"sorted_exp_1to2.npy",sorted_exp) #(ngenes,ntimes)
np.save(outpath+freq+"gt_adj_1to2.npy",gt_adj_time) #(ntimes,ngenes,ngenes)